In [17]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder

def evaluate_model(y_true, y_pred):
    overall_acc = accuracy_score(y_true, y_pred)
    report = classification_report(y_true, y_pred, output_dict=True, zero_division=0)

    # Extract per-class accuracy using numeric keys directly from the report
    per_class_acc = {
        cls: report[str(cls)]['recall']  # Use string of the class number (e.g., '0', '1', '2')
        for cls in np.unique(y_true)  # This ensures we use all class labels present in y_true
    }
    return overall_acc, per_class_acc

root_dir = 'data-processed/Experiment 3 Data'
results = {}

for challenge in os.listdir(root_dir):
    if challenge != "challenge-beginners-2018":
        continue
    challenge_path = os.path.join(root_dir, challenge)
    if not os.path.isdir(challenge_path):
        continue

    challenge_results = {'LogReg': [], 'RF': [], 'SVM': [], 'DT': []}

    for file in os.listdir(challenge_path):
        if not file.endswith('.csv'):
            continue

        df = pd.read_csv(os.path.join(challenge_path, file))

        if 'Outcome' not in df.columns:
            continue

        # Assume the 'slide' columns are named like 'slide1', 'slide2', ..., 'slideN'
        slide_cols = [col for col in df.columns if col.startswith('slide')]
        X_raw = df[slide_cols]
        y_raw = df['Outcome']

        # Convert event slides (Zero/One) and problem slides (class0, class1, class2) to numeric
        for col in X_raw.columns:
            try:
                X_raw[col] = pd.to_numeric(X_raw[col])
                print(col)
            except ValueError:
                continue  # Leave as is for one-hot encoding later

        # One-hot encode categorical columns for problem slides (class0, class1, class2)
        X = pd.get_dummies(X_raw)

        # Drop rows with any missing data
        mask = X.notnull().all(axis=1) & y_raw.notnull()
        X = X[mask]
        y = y_raw[mask]

        if X.shape[0] < 10:
            print(f"Skipping {file} in {challenge} (too few samples after cleanup).")
            continue

        encoder = LabelEncoder()
        y_enc = encoder.fit_transform(y)

        if len(np.unique(y_enc)) < 2:
            print(f"Skipping {file} in {challenge} (only one class present).")
            continue

        skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
        fold_metrics = {'LogReg': [], 'RF': [], 'SVM': [], 'DT': []}

        for train_idx, test_idx in skf.split(X, y_enc):
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y_enc[train_idx], y_enc[test_idx]

            # Logistic Regression
            logreg = LogisticRegression(max_iter=1000)
            logreg.fit(X_train, y_train)
            acc, per_class = evaluate_model(y_test, logreg.predict(X_test))
            fold_metrics['LogReg'].append((acc, per_class))

            # Random Forest
            rf = RandomForestClassifier(n_estimators=100)
            rf.fit(X_train, y_train)
            acc, per_class = evaluate_model(y_test, rf.predict(X_test))
            fold_metrics['RF'].append((acc, per_class))

            # SVM
            svm = SVC()
            svm.fit(X_train, y_train)
            acc, per_class = evaluate_model(y_test, svm.predict(X_test))
            fold_metrics['SVM'].append((acc, per_class))

            # Decision Tree (default)
            dt = DecisionTreeClassifier()
            dt.fit(X_train, y_train)
            acc, per_class = evaluate_model(y_test, dt.predict(X_test))
            fold_metrics['DT'].append((acc, per_class))

        for model in fold_metrics:
            accs = [m[0] for m in fold_metrics[model]]
            per_class_all = [m[1] for m in fold_metrics[model]]
            avg_acc = np.mean(accs)

            # Calculate per-class accuracy for class0, class1, class2 only
            avg_class_acc = {
                k: np.mean([d.get(k, 0) for d in per_class_all]) for k in [1, 2, 3]
            }

            challenge_results[model].append((avg_acc, avg_class_acc))

    # Average across all files in the challenge
    results[challenge] = {}
    for model in challenge_results:
        accs = [r[0] for r in challenge_results[model]]
        per_class_all = [r[1] for r in challenge_results[model]]
        avg_acc = np.mean(accs) if accs else 0

        avg_class_acc = {
            k: np.mean([d.get(k, 0) for d in per_class_all]) for k in ['class0', 'class1', 'class2']
        }

        results[challenge][model] = {
            'Overall Accuracy': avg_acc,
            'Per Class Accuracy': avg_class_acc
        }

# Print report
for challenge, models in results.items():
    print(f"\n=== Challenge: {challenge} ===")
    for model, metrics in models.items():
        print(f"\nModel: {model}")
        print(f"  Overall Accuracy: {metrics['Overall Accuracy']:.3f}")
        print("  Per Class Accuracy:")
        for cls, acc in metrics['Per Class Accuracy'].items():
            print(f"    {cls}: {acc:.3f}")


C:\Users\vince\AppData\Local\Temp\ipykernel_3828\3906806925.py:39: DtypeWarning: Columns (18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(challenge_path, file))
C:\Users\vince\AppData\Local\Temp\ipykernel_3828\3906806925.py:39: DtypeWarning: Columns (18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(challenge_path, file))
C:\Users\vince\AppData\Local\Temp\ipykernel_3828\3906806925.py:39: DtypeWarning: Columns (18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(challenge_path, file))
C:\Users\vince\AppData\Local\Temp\ipykernel_3828\3906806925.py:39: DtypeWarning: Columns (18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(challenge_path, file))
C:\Users\vince\AppData\Local\Temp\ipykernel_3828\3906806925.py:39: DtypeWarning: Col


=== Challenge: challenge-beginners-2018 ===

Model: LogReg
  Overall Accuracy: 0.946
  Per Class Accuracy:
    class0: 0.000
    class1: 0.000
    class2: 0.000

Model: RF
  Overall Accuracy: 0.942
  Per Class Accuracy:
    class0: 0.000
    class1: 0.000
    class2: 0.000

Model: SVM
  Overall Accuracy: 0.945
  Per Class Accuracy:
    class0: 0.000
    class1: 0.000
    class2: 0.000

Model: DT
  Overall Accuracy: 0.926
  Per Class Accuracy:
    class0: 0.000
    class1: 0.000
    class2: 0.000
